# Ising Model

In [ ]:
using Pkg
Pkg.activate("../")

In [ ]:
using Plots
using FLoops

In [64]:
include("../src/spinmc.jl");

In [ ]:
@time temps, mags = let L = 16
    temps = collect(0.5:0.1:3.5)
    mags = zeros(Float64, length(temps))
    nsteps = 1000
    Threads.@threads for t_idx in eachindex(temps)
        m = 0
        model = ClassicalIsingModel2D(L, :cold)
        stack = LazyStack(CartesianIndex{2})
        cluster = falses(L, L)
        T = temps[t_idx]
        P_add = isingwolff_Padd(T)
        for i=1:nsteps
            wolff_update!(model, T, cluster, stack; P_add=P_add)
            m += magnetization(model)
        end
        mags[t_idx] = m / (nsteps*model.L^2) 
    end
    temps, mags
end

In [ ]:
scatter(temps, mags, xlabel="Temperature", ylabel="Magnetization", title="Ising Model")

In [ ]:
cls = falses(10, 10)
mm = ClassicalIsingModel2D(10, :cold)
wolff_update!(mm, 0.1, cls)

In [ ]:
using BenchmarkTools

In [ ]:
ff = falses(10, 10)

function make_false!(m::BitMatrix)
    m .= false
end

@btime ff .= true
@btime for i=1:100
    make_false!(ff)
end

In [65]:
@time ising_get_measurements_to_txt(24, 2.2, 1000, 1000, 10,
    ntau=2,
    start=:cold,
    store_at="D:\\Projects\\Dr. Heyl Group\\montecarlo-spin-configurations-and-distances-study\\data",
    verbose=false,
    mode="w",
    get_mags=true,
    get_structure_factors=true)

 30.474274 seconds (154.44 M allocations: 4.175 GiB, 3.94% gc time, 0.50% compilation time)


In [27]:
let
    data_path = "D:\\Projects\\Dr. Heyl Group\\montecarlo-spin-configurations-and-distances-study\\data"
    data = readdlm(joinpath([data_path, "Ising", "Size64", "uncorr_configs", "ising_uncorr_configs_temp2.2_L64.txt"]), ',', Int)
    println(size(data))
end

(4096, 1000)


In [63]:
let 
    mm = ClassicalIsingModel2D(48, :hot)
    for i=1:100
        wolff_update!(mm, 2.4)
    end
    @assert structure_factor(mm) == structure_factor(mm.lattice, 48)
    println(structure_factor(mm))
    println(structure_factor(mm.lattice, 48))
    @time structure_factor(mm)
    @time structure_factor(mm.lattice, 48)
end

0.006658106674382716
0.006658106674382716
  0.000000 seconds
  0.003481 seconds


0.006658106674382716